In [1]:
import syft as sy

In [2]:
import statsmodels
import re

[2021-06-13T10:01:56.225772+0530][CRITICAL][logger]][21697] Unable to load package support for: statsmodels. 'NoneType' object has no attribute 'update'


In [3]:
sy.lib_ast

Module:
	.syft -> Module:
		.lib -> Module:
			.python -> Module:
				.collections -> Module:
					.OrderedDict -> <syft.ast.klass.Class object at 0x7fad65178a60>

				.Bool -> <syft.ast.klass.Class object at 0x7fad651784c0>
				.Complex -> <syft.ast.klass.Class object at 0x7fad65178520>
				.Dict -> <syft.ast.klass.Class object at 0x7fad65178580>
				.Float -> <syft.ast.klass.Class object at 0x7fad651785e0>
				.Int -> <syft.ast.klass.Class object at 0x7fad65178640>
				.List -> <syft.ast.klass.Class object at 0x7fad651786a0>
				.Slice -> <syft.ast.klass.Class object at 0x7fad65178700>
				.Range -> <syft.ast.klass.Class object at 0x7fad65178760>
				.String -> <syft.ast.klass.Class object at 0x7fad651787c0>
				._SyNone -> <syft.ast.klass.Class object at 0x7fad65178820>
				.PyPrimitive -> <syft.ast.klass.Class object at 0x7fad65178880>
				.Any -> <syft.ast.klass.Class object at 0x7fad651788e0>
				.Tuple -> <syft.ast.klass.Class object at 0x7fad65178940>
				.Iterator -> <syft.ast

In [4]:
import syft_statsmodels

Adding support for statsmodels


In [5]:
import pandas as pd

In [6]:
root_client = sy.VirtualMachine().get_root_client()

In [7]:
FAMILY = [
    statsmodels.genmod.families.Binomial,
    statsmodels.genmod.families.Gamma,
    statsmodels.genmod.families.Gaussian,
    statsmodels.genmod.families.InverseGaussian,
    statsmodels.genmod.families.NegativeBinomial,
    statsmodels.genmod.families.Poisson,
    statsmodels.genmod.families.Tweedie,
]

UNNECESSARY_STR = r"Time(.*)(?=Pearson)|Date(.*)(?=Deviance)"

# sy.load("pandas")
# sy.load("statsmodels")

# load data
url = "https://raw.githubusercontent.com/chemo-wakate/tutorial-6th/master/beginner/data/winequality-red.txt"
df = pd.read_csv(url, sep="\t")
df["quality"] = df["quality"].apply(lambda x: 1 if x >= 6 else 0)
df = df.sample(100, random_state=42)
df_ptr = df.send(root_client)

# explanatory variable
x = df["fixed acidity"]
x_ptr = df_ptr["fixed acidity"]

# add constant
_x = statsmodels.api.add_constant(x)
_x_ptr = root_client.statsmodels.api.add_constant(x_ptr)

# dependent variable
_y = df["quality"]
_y_ptr = df_ptr["quality"]

# test all possible combinations of families and links
for family in FAMILY:
    for link in family.links:

        print(family, link)

        model = statsmodels.genmod.generalized_linear_model.GLM(
            _y, _x, family=family(link=link())
        )
        result = model.fit()
        summary = result.summary().as_csv()

        remote_model = root_client.statsmodels.genmod.generalized_linear_model.GLM(
            _y_ptr, _x_ptr, family=family(link=link())
        )
        remote_result = remote_model.fit()
        # `get` corresponds to `summary().as_csv()`
        remote_summary = remote_result.get()

        # remove unnnecessary strings such as proccesing time and date
        summary = re.sub(UNNECESSARY_STR, "", summary)
        remote_summary = re.sub(UNNECESSARY_STR, "", remote_summary)

        assert summary == remote_summary


<class 'statsmodels.genmod.families.family.Binomial'> <class 'statsmodels.genmod.families.links.logit'>
<class 'statsmodels.genmod.families.family.Binomial'> <class 'statsmodels.genmod.families.links.probit'>
<class 'statsmodels.genmod.families.family.Binomial'> <class 'statsmodels.genmod.families.links.cauchy'>
<class 'statsmodels.genmod.families.family.Binomial'> <class 'statsmodels.genmod.families.links.log'>
<class 'statsmodels.genmod.families.family.Binomial'> <class 'statsmodels.genmod.families.links.cloglog'>
<class 'statsmodels.genmod.families.family.Binomial'> <class 'statsmodels.genmod.families.links.identity'>
<class 'statsmodels.genmod.families.family.Gamma'> <class 'statsmodels.genmod.families.links.log'>
<class 'statsmodels.genmod.families.family.Gamma'> <class 'statsmodels.genmod.families.links.identity'>
<class 'statsmodels.genmod.families.family.Gamma'> <class 'statsmodels.genmod.families.links.inverse_power'>
<class 'statsmodels.genmod.families.family.Gaussian'> <clas

/home/alpha91/miniconda3/envs/torch/lib/python3.9/site-packages/statsmodels/genmod/generalized_linear_model.py:293: DomainWarning: The log link function does not respect the domain of the Binomial family.
  warnings.warn((f"The {type(family.link).__name__} link function "
/home/alpha91/miniconda3/envs/torch/lib/python3.9/site-packages/statsmodels/genmod/generalized_linear_model.py:293: DomainWarning: The log link function does not respect the domain of the Binomial family.
  warnings.warn((f"The {type(family.link).__name__} link function "
/home/alpha91/miniconda3/envs/torch/lib/python3.9/site-packages/statsmodels/genmod/generalized_linear_model.py:293: DomainWarning: The identity link function does not respect the domain of the Binomial family.
  warnings.warn((f"The {type(family.link).__name__} link function "
/home/alpha91/miniconda3/envs/torch/lib/python3.9/site-packages/statsmodels/genmod/generalized_linear_model.py:293: DomainWarning: The identity link function does not respect t

<class 'statsmodels.genmod.families.family.Gaussian'> <class 'statsmodels.genmod.families.links.inverse_power'>
<class 'statsmodels.genmod.families.family.InverseGaussian'> <class 'statsmodels.genmod.families.links.inverse_squared'>
<class 'statsmodels.genmod.families.family.InverseGaussian'> <class 'statsmodels.genmod.families.links.inverse_power'>
<class 'statsmodels.genmod.families.family.InverseGaussian'> <class 'statsmodels.genmod.families.links.identity'>
<class 'statsmodels.genmod.families.family.InverseGaussian'> <class 'statsmodels.genmod.families.links.log'>
<class 'statsmodels.genmod.families.family.NegativeBinomial'> <class 'statsmodels.genmod.families.links.log'>
<class 'statsmodels.genmod.families.family.NegativeBinomial'> <class 'statsmodels.genmod.families.links.cloglog'>
<class 'statsmodels.genmod.families.family.NegativeBinomial'> <class 'statsmodels.genmod.families.links.identity'>
<class 'statsmodels.genmod.families.family.NegativeBinomial'> <class 'statsmodels.genm

/home/alpha91/miniconda3/envs/torch/lib/python3.9/site-packages/statsmodels/genmod/families/family.py:1077: RuntimeWarning: divide by zero encountered in true_divide
  return 1. / (endog * mu ** 2) * (endog - mu) ** 2
/home/alpha91/miniconda3/envs/torch/lib/python3.9/site-packages/statsmodels/genmod/families/family.py:1109: RuntimeWarning: divide by zero encountered in true_divide
  ll_obs = -var_weights * (endog - mu) ** 2 / (scale * endog * mu ** 2)
/home/alpha91/miniconda3/envs/torch/lib/python3.9/site-packages/statsmodels/genmod/families/family.py:1110: RuntimeWarning: divide by zero encountered in log
  ll_obs += -np.log(scale * endog ** 3 / var_weights) - np.log(2 * np.pi)
/home/alpha91/miniconda3/envs/torch/lib/python3.9/site-packages/statsmodels/genmod/families/family.py:1110: RuntimeWarning: invalid value encountered in add
  ll_obs += -np.log(scale * endog ** 3 / var_weights) - np.log(2 * np.pi)
/home/alpha91/miniconda3/envs/torch/lib/python3.9/site-packages/statsmodels/genmo